In [4]:
import pandas as pd
import numpy as np
import os
import time
import requests
from dotenv import load_dotenv
from tqdm import tqdm

# Loading OneMap token
load_dotenv()
ONEMAP_API_TOKEN = os.getenv("ONEMAP_API_TOKEN")

In [5]:
resale_1990_1999 = pd.read_csv('./data/raw_data/resale_1990_1999.csv')
resale_2000_2012 = pd.read_csv('./data/raw_data/resale_2000_2012feb.csv')
resale_2012_2014 = pd.read_csv('./data/raw_data/resale_2012mar_2014dec.csv')
resale_2015_2016 = pd.read_csv('./data/raw_data/resale_2015jan_2016dec.csv')
resale_2017_2025 = pd.read_csv('./data/raw_data/resale_2017_2025.csv')

In [6]:
print(resale_1990_1999.columns)
print(resale_2000_2012.columns)
print(resale_2012_2014.columns)
print(resale_2015_2016.columns)
print(resale_2017_2025.columns)

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')


There is an extra column 'remaining_lease' from years 2015 onwards. To handle this, we can either do one of the following:
<ol>
<li>Calculate the remaining lease for the previous years and append that to the previous years' dataframe.</li>
<li>Drop the remaining lease column for 2015 onwards</li>
<li>Use only data from 2015 onwards</li>
</ol>

<b>Option 3</b> was chosen for two reasons:
- In context, remaining number of years of a lease is likely a significant factor of consideration for home buyers, hence it would not make sense to drop this column.
- Between years 2015 and 2025, there is sufficient data for us to work with (~250k records) for prediction.

Hence, we will only be making use of data from 2015 onwards.

<h2>Checking for NaN</h2>

In [7]:
print(resale_2015_2016.isnull().any())
print(resale_2017_2025.isnull().any())

month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
dtype: bool
month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
dtype: bool


We note that there are no missing values in the data from 2015 to 2025, which is a good start. There are no rows for us to drop nor fill in.

<h2>Checking dtypes</h2>

In [8]:
resale_2015_2016.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease          int64
resale_price           float64
dtype: object

In [9]:
resale_2017_2025.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object

Note that remaining_lease is of a different dtype for both dataframes. This is because the format from 2017 onwards captures both the number of years and months left, while in 2015-2016 only the number of years is captured.

To standardize this, we will round down the 2017–2025 values to the nearest whole year and convert the column to int64 to represent the remaining lease in full years.

In [10]:
# Removing months from remaining_lease for 2017-2025 to nearest floor by year
resale_2017_2025['remaining_lease'] = resale_2017_2025['remaining_lease'].str.extract(r'(\d+)\s+years').astype(int)

In [11]:
df = pd.concat([resale_2015_2016, resale_2017_2025], ignore_index=True)
df.shape

(249323, 11)

<h1>Feature Engineering</h2>

Here, we will be attempting to bring in distance based features to local amenities and facilities as these are factors that play major factor when home buyers consider a house.

We will be engineering the features that capture the block's distance to nearest hawker centre, school, hospital, childcare, kindergarten, park, community club, and distance to Downtown Core. We will also be capturing the number of each of these amenities within 1km of the block.

To do so, we deploy the help of OneMapAPI, as well as data that we could get from data.gov.sg.

<h2>Geocoding Blocks</h2>

We will first begin by geocoding the blocks to find their coordinates, based on OneMapAPI's search method.

In [12]:
# Address -> (lat, lon) cache
address_cache = {}

# Helper function to call OneMapAPI's Search method
def geocode_address(address):
    if address in address_cache:
        return address_cache[address]
    
    url = "https://www.onemap.gov.sg/api/common/elastic/search"
    headers = {"Authorization": f"Bearer {ONEMAP_API_TOKEN}"}
    params = {
        'searchVal': address,
        'returnGeom': 'Y',
        'getAddrDetails': 'Y',
        'pageNum': 1
    }

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        time.sleep(0.5)
        if response.status_code == 200:
            results = response.json().get("results")
            if results:
                lat = float(results[0]['LATITUDE'])
                lon = float(results[0]['LONGITUDE'])
                address_cache[address] = (lat, lon)
                return (lat, lon)
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Address: {address} | {e}")
    address_cache[address] = (None, None)
    return (None, None)

# Helper function to call geocode_address in batches to avoid timeouts, on top of sleeping in geocode_address.
def batch_geocode(df, address_column='full_address'):
    coords = []
    for addr in tqdm(df[address_column]):
        coords.append(geocode_address(addr))
    return coords


In [13]:
# Creating full_address column by concatenating block + street_name
df['full_address'] = df['block'].astype(str) + ' ' + df['street_name']

In [ ]:
# Drop duplicate addresses to reduce API calls
df_unique = df[['full_address']].drop_duplicates().reset_index(drop=True)

# Run batch geocoding
# WARNING!!!!! This took me 3.5 hours to run.
df_unique['coordinates'] = batch_geocode(df_unique, 'full_address')

 67%|██████▋   | 6446/9682 [2:38:03<9:14:27, 10.28s/it]   

[ERROR] Address: 146 RIVERVALE DR | HTTPSConnectionPool(host='www.onemap.gov.sg', port=443): Read timed out. (read timeout=10)


 78%|███████▊  | 7590/9682 [2:51:55<2:00:13,  3.45s/it]

[ERROR] Address: 308 TAMPINES ST 32 | HTTPSConnectionPool(host='www.onemap.gov.sg', port=443): Read timed out. (read timeout=10)


100%|██████████| 9682/9682 [3:16:09<00:00,  1.22s/it]  


In [ ]:
# Merge coordinates back to the original dataframe
df = df.merge(df_unique, on='full_address', how='left')

# Split into lat/lon
df[['latitude', 'longitude']] = pd.DataFrame(df['coordinates'].tolist(), index=df.index)

In [31]:
print(df.isnull().any())

month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
full_address           False
coordinates            False
dtype: bool


<h3>Distance to nearest MRT Station & Number of MRT Stations within 1 km from the block.</h3>

To do so, we will be using data scraped from https://spotters.sgtrains.com/guide-abbreviation, geocode the MRT Stations with OneMAPAPI, and then calculate the distance between the MRT station and the block using scipy.spatial's cKDTree module.

In [ ]:
from scipy.spatial import cKDTree

# Load MRT geocoded stations
df_mrt = pd.read_csv("geocoded_mrt_stations.csv")
mrt_coords = df_mrt[['latitude', 'longitude']].to_numpy()

# Drop rows with missing or infinite coordinates
df = df.dropna(subset=['latitude', 'longitude'])
df = df[np.isfinite(df['latitude']) & np.isfinite(df['longitude'])]

# Now extract coordinates safely
hdb_coords = df[['latitude', 'longitude']].to_numpy()

# Build cKDTree for fast spatial lookup
mrt_tree = cKDTree(mrt_coords)

# Conversion factor: degrees to kilometers
DEGREE_TO_KM = 111  # ~111km per degree on Earth's surface

#Compute distance to nearest MRT
distances, _ = mrt_tree.query(hdb_coords, k=1)
df['distance_to_nearest_mrt_km'] = (distances * DEGREE_TO_KM).round(5)

#Compute number of MRT stations within 1km
radius_km = 1
radius_degrees = radius_km / DEGREE_TO_KM
neighbors_within_1km = mrt_tree.query_ball_point(hdb_coords, r=radius_degrees)
df['num_mrt_within_1km'] = [len(neighbors) for neighbors in neighbors_within_1km]

<h3>Fetch OneMap themes</h3>

Here, we will be using OneMapAPI's theme method to get the necessary themes for each amenity listed earlier, to calculate the distance to them.

In [ ]:
import requests
import pandas as pd

def fetch_amenity_coordinates(queryname: str, token: str) -> pd.DataFrame:
    """
    Fetch amenity point data from OneMap Theme API based on queryname.
    Extracts latitude and longitude from the 'LatLng' field.
    """
    url = f"https://www.onemap.gov.sg/api/public/themesvc/retrieveTheme?queryName={queryname}"
    headers = {"Authorization": token}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    srch_results = data.get("SrchResults", [])[1:]

    records = []
    for item in srch_results:
        try:
            latlng = item.get("LatLng")
            if latlng:
                lat_str, lon_str = latlng.strip().split(",")
                lat = float(lat_str)
                lon = float(lon_str)
                name = item.get("NAME", queryname)
                records.append({"name": name, "latitude": lat, "longitude": lon})
        except (ValueError, AttributeError):
            continue

    return pd.DataFrame(records)

def add_distance_features(df_hdb: pd.DataFrame, df_amenity: pd.DataFrame,
                          amenity_label: str, radius_km: float = 1.0) -> pd.DataFrame:
    """
    Adds distance to nearest amenity and count within radius_km.
    """
    # Build tree
    amenity_coords = df_amenity[['latitude', 'longitude']].to_numpy()
    hdb_coords = df_hdb[['latitude', 'longitude']].to_numpy()
    tree = cKDTree(amenity_coords)

    DEGREE_TO_KM = 111
    radius_deg = radius_km / DEGREE_TO_KM

    # Distance to nearest
    distances, _ = tree.query(hdb_coords, k=1)
    df_hdb[f'distance_to_nearest_{amenity_label}_km'] = (distances * DEGREE_TO_KM).round(5)

    # Count within radius
    neighbors = tree.query_ball_point(hdb_coords, r=radius_deg)
    df_hdb[f'num_{amenity_label}s_within_{int(radius_km)}km'] = [len(n) for n in neighbors]

    return df_hdb


In [ ]:
# Hawker Centres
df_hawkers = fetch_amenity_coordinates("ssot_hawkercentres", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_hawkers, amenity_label="hawker_centre")

# Libraries
df_libraries = fetch_amenity_coordinates("libraries", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_libraries, amenity_label="library")

# Hospitals
df_hospitals = fetch_amenity_coordinates("moh_hospitals", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_hospitals, amenity_label="hospital")

# Childcare
df_childcares = fetch_amenity_coordinates("childcare", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_childcares, amenity_label="childcare")

# Community Clubs
df_kindergartens = fetch_amenity_coordinates("kindergartens", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_kindergartens, amenity_label="kindergarten")

# Parks
df_parks = fetch_amenity_coordinates("nationalparks", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_parks, amenity_label="park")

# Community Clubs
df_cc = fetch_amenity_coordinates("communityclubs", ONEMAP_API_TOKEN)
df = add_distance_features(df_hdb=df, df_amenity=df_cc, amenity_label="communityclub")

<h3>Distance to nearest school and number of schools within 1km</h3>

Here, we do the same approach for schools. The data is retrieved from https://data.gov.sg/datasets/d_688b934f82c1059ed0a6993d2a829089/view, that provides the general information of primary, secondary and pre-university schools. In which, we can find the postal code of the schools and use OneMapAPI's search method to geocode it.

Lastly, we calculate the distance again using cKDTree, which is found in add_distance_features() we had defined earlier.

In [ ]:
def geocode_postal_codes(df: pd.DataFrame, token: str, postal_col: str = "postal_code") -> pd.DataFrame:

    geocoded = []

    for postal_code_raw in df[postal_col].astype(str).unique():
        postal_code = postal_code_raw.zfill(6)  # pad to 6 digits to prevent missing start 0 for postal codes that begin with 0

        url = f"https://www.onemap.gov.sg/api/common/elastic/search"
        params = {
            "searchVal": postal_code,
            "returnGeom": "Y",
            "getAddrDetails": "Y",
            "pageNum": 1
        }
        headers = {"Authorization": token}

        try:
            resp = requests.get(url, params=params, headers=headers)
            resp.raise_for_status()
            results = resp.json().get("results", [])
            if results:
                lat = float(results[0]["LATITUDE"])
                lon = float(results[0]["LONGITUDE"])
                geocoded.append({"postal_code": postal_code, "latitude": lat, "longitude": lon})
        except Exception as e:
            print(f"Failed to geocode postal code {postal_code}: {e}")
        
        time.sleep(0.1)

    geo_df = pd.DataFrame(geocoded)
    df["postal_code"] = df["postal_code"].astype(str).str.zfill(6)
    return df.merge(geo_df, on="postal_code", how="left")


In [93]:
# List of schools and information
schools_df = pd.read_csv('Generalinformationofschools.csv')

# Geocode schools by postal code
geocoded_schools_df = geocode_postal_codes(schools_df, token=ONEMAP_API_TOKEN)

print(geocoded_schools_df[['school_name', 'postal_code', 'latitude', 'longitude']].head())

                      school_name postal_code  latitude   longitude
0        ADMIRALTY PRIMARY SCHOOL      738907  1.442635  103.800040
1      ADMIRALTY SECONDARY SCHOOL      737916  1.445891  103.802398
2    AHMAD IBRAHIM PRIMARY SCHOOL      768643  1.433153  103.832942
3  AHMAD IBRAHIM SECONDARY SCHOOL      768928  1.436060  103.829719
4                  AI TONG SCHOOL      579646  1.360583  103.833020


In [96]:
df = add_distance_features(df_hdb=df, df_amenity=geocoded_schools_df, amenity_label="school")

<h3>Distance to Downtown Core</h3>

We will be including the distance to the Downtown Core area of Singapore (distance_to_downtown_core_km), since it is another significant factor of consideration for home buyers. As the downtown core is an area, we will be using the coordinates of Raffles Place MRT as the centroid of downtown core to calculate this distance.

In [33]:
# Raffles Place MRT (approx.)
downtown_core_coords = (1.2831, 103.8515)

In [36]:
from scipy.spatial import distance

def add_distance_to_point(df, point_coords, label='downtown_core'):
    DEGREE_TO_KM = 111
    df[f'distance_to_{label}_km'] = df.apply(
        lambda row: distance.euclidean((row['latitude'], row['longitude']), point_coords) * DEGREE_TO_KM,
        axis=1
    ).round(5)
    return df

# Apply to df
df = add_distance_to_point(df, downtown_core_coords, label='downtown_core')

In [ ]:
df.to_csv('final_dataset.csv', index=False)

The full dataset now contains the new features of distance to various amenities, number of such amenities within 1km, and the distance to downtore core. This dataset is saved to 'final_dataset.csv'.

In the following section, we will attempt to preprocess the dataset, then create two copies of the dataset - one for linear modelling, another for tree-based modelling, before encoding the necessary attributes in appropriate ways.

<h1>Data Preprocessing</h1>

In [ ]:
# Convert month to datetime
df['month'] = pd.to_datetime(df['month'])

# Extract year and month
df['year'] = df['month'].dt.year
df['month_num'] = df['month'].dt.month

# Drop columns not useful for modelling
df = df.drop(columns=['block', 'street_name', 'full_address', 'coordinates', 'month'])

<h3>Creating df_linear for linear modelling</h3>

For categorical attributes, we will be encoding them in various methods:
- One-hot encoding is selected for 'town' and 'flat_model', as these attributes do not have meaningful order in the categories.
- Ordinal encoding is performed for 'flat_type', as there is meaningful order in terms of number fo rooms and mean square area. We will be encoding them to values ranging from 1 to 7 (1-5 for 1-5 ROOMS, 6 for 'Executive' and 7 for 'Multi-Generation').
- For 'storey_range', we will be taking the median of the range to convert it from a categorical attribute.

In [45]:
df_linear = df.copy()

# One-hot encoding for town and flat_model, since they do not have meaningful order
categorical_cols = ['town', 'flat_model']
df_linear = pd.get_dummies(df_linear, columns=categorical_cols, drop_first=True)

# Ordinal encoding for flat_type, as there is meaningful order in terms of number of rooms and mean square area.
flat_type_order = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI-GENERATION': 7
}
df['flat_type_encoded'] = df['flat_type'].map(flat_type_order)
df_linear.drop('flat_type', axis=1, inplace=True)

# Story Median - we shall extract the median of storey_range to convert it from categorical to numerical.
def extract_storey_median(storey_str):
    try:
        low, high = map(int, storey_str.split(' TO '))
        return (low + high) / 2
    except:
        return np.nan  # handle unexpected formatting
df_linear['storey_median'] = df_linear['storey_range'].apply(extract_storey_median)
df_linear.drop('storey_range', axis=1, inplace=True)

# Remove datetime and non-numeric fields if still present
non_numeric_cols = df_linear.select_dtypes(include=["datetime64", "object"]).columns
df_linear = df_linear.drop(columns=non_numeric_cols)

In [46]:
df_linear.to_csv('dataset_for_linear.csv', index=False)

<h3>Creating df_tree for tree-based models</h3>

For tree-based models like Random Forest, XGBoost, LightGBM, CatBoost, they can handle ordinal like numbers without assuming linear relationships. In fact, one-hot encoding might even reduce performance due to unnecessary feature splits.

As such, we will simply be using LabelEncoder from sklearn to encode the categorical attributes.

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_tree = df.copy()
encoders = {}

for col in ['town', 'flat_type', 'flat_model', 'storey_range']:
    le = LabelEncoder()
    df_tree[col] = le.fit_transform(df_tree[col])
    encoders[col] = le

    # Print mapping
    print(f"Encoding for '{col}':")
    for category, code in zip(le.classes_, le.transform(le.classes_)):
        print(f"  {category} → {code}")
    print()

# Drop unused columns
df_tree = df_tree.drop(columns=['month', 'coordinates', 'block', 'street_name', 'full_address'], errors='ignore')

Encoding for 'town':
  ANG MO KIO → 0
  BEDOK → 1
  BISHAN → 2
  BUKIT BATOK → 3
  BUKIT MERAH → 4
  BUKIT PANJANG → 5
  BUKIT TIMAH → 6
  CENTRAL AREA → 7
  CHOA CHU KANG → 8
  CLEMENTI → 9
  GEYLANG → 10
  HOUGANG → 11
  JURONG EAST → 12
  JURONG WEST → 13
  KALLANG/WHAMPOA → 14
  MARINE PARADE → 15
  PASIR RIS → 16
  PUNGGOL → 17
  QUEENSTOWN → 18
  SEMBAWANG → 19
  SENGKANG → 20
  SERANGOON → 21
  TAMPINES → 22
  TOA PAYOH → 23
  WOODLANDS → 24
  YISHUN → 25

Encoding for 'flat_type':
  1 ROOM → 0
  2 ROOM → 1
  3 ROOM → 2
  4 ROOM → 3
  5 ROOM → 4
  EXECUTIVE → 5
  MULTI-GENERATION → 6

Encoding for 'flat_model':
  2-room → 0
  3Gen → 1
  Adjoined flat → 2
  Apartment → 3
  DBSS → 4
  Improved → 5
  Improved-Maisonette → 6
  Maisonette → 7
  Model A → 8
  Model A-Maisonette → 9
  Model A2 → 10
  Multi Generation → 11
  New Generation → 12
  Premium Apartment → 13
  Premium Apartment Loft → 14
  Premium Maisonette → 15
  Simplified → 16
  Standard → 17
  Terrace → 18
  Type S1 → 19

<h3>Saving encoder for future use</h3>

So that we can use it for mapping to display original values as selections in the frontend for UI/UX purposes.

In [ ]:
import joblib

# Save all encoders as a single .pkl file
joblib.dump(encoders, './model/label_encoders.pkl')

['label_encoders.pkl']

<h1>Conclusion</h1>

We now have 3 sets of data:
1. Full Dataset (final_dataset.csv): contains data from 2015-2025, preprocessed and added features.
2. Dataset for Linear Modelling (dataset_for_linear.csv): dataset to use for linear models, one-hot and ordinally encoded.
3. Dataset for Tree-based Modelling (dataset_for_tree.csv): dataset to use for tree models, encoded with sklearn's LabelEncoder.

Next, we will be building and running models on the datasets in Models.ipynb.